In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

# Indicator 3.2

### Load data

In [3]:
districts = pd.read_excel('../data/raw/sws-data.xlsx', 'districts')
districts.head()

,geography,district,concept
0,Kenya,Kitui County,4
1,Uganda,Kabarole Learning Alliance,1
2,Uganda,Kumi District,3
3,Uganda,Nakaseke District,3
4,Uganda,Kamuli District,3


In [4]:
pms = pd.read_excel('../data/raw/sws-data.xlsx', 'pms')
pms.head()

,district,pm_cat,pm_id,pm_text
0,Kitui County,Expect to See,1,NaN
1,Kitui County,Expect to See,2,NaN
2,Kitui County,Expect to See,3,NaN
3,Kitui County,Like to See,4,NaN
4,Kitui County,Like to See,5,NaN


In [5]:
pm_data = pd.read_excel('../data/raw/sws-data.xlsx', 'pm_data')
pm_data.head()

,district,pm_id,year,quarter,result
0,Kitui County,1,2018,2,1.0
1,Kitui County,1,2018,4,2.0
2,Kitui County,1,2019,2,2.0
3,Kitui County,1,2019,4,2.0
4,Kitui County,2,2018,2,1.0


In [6]:
df_temp = pd.merge(pm_data, pms, on=['district', 'pm_id'], how='left')
df = pd.merge(df_temp, districts, on='district', how='left')
df.head()

,district,pm_id,year,quarter,result,pm_cat,pm_text,geography,concept
0,Kitui County,1,2018,2,1.0,Expect to See,NaN,Kenya,4
1,Kitui County,1,2018,4,2.0,Expect to See,NaN,Kenya,4
2,Kitui County,1,2019,2,2.0,Expect to See,NaN,Kenya,4
3,Kitui County,1,2019,4,2.0,Expect to See,NaN,Kenya,4
4,Kitui County,2,2018,2,1.0,Expect to See,NaN,Kenya,4


### Average result by PM category

In [7]:
results = df.groupby(['concept', 'district', 'pm_cat', 'year', 'quarter']).mean()[['result']]
results

result
concept district     pm_cat        year quarter          
1       Debre Birhan Expect to See 2019 2        2.666667
                                        4        2.666667
                     Like to See   2019 2        1.500000
                                        4        2.000000
                     Love to See   2019 2        1.666667
...                                                   ...
4       Kitui County Like to See   2019 4        1.333333
                     Love to See   2018 2        1.000000
                                        4        1.000000
                                   2019 2        1.000000
                                        4        1.000000

[90 rows x 1 columns]

#### How to slice the results dataframe with a multi-index

In [8]:
idx = pd.IndexSlice
results.loc[idx[:, 'Debre Birhan', 'Expect to See'], :]

result
concept district     pm_cat        year quarter          
1       Debre Birhan Expect to See 2019 2        2.666667
                                        4        2.666667

#### Or, reset the index to remove multi-index

In [9]:
results_tidy = results.reset_index(drop=False)
results_tidy.head()

,concept,district,pm_cat,year,quarter,result
0,1,Debre Birhan,Expect to See,2019,2,2.666667
1,1,Debre Birhan,Expect to See,2019,4,2.666667
2,1,Debre Birhan,Like to See,2019,2,1.500000
3,1,Debre Birhan,Like to See,2019,4,2.000000
4,1,Debre Birhan,Love to See,2019,2,1.666667


In [10]:
results_tidy.groupby(['concept', 'year', 'quarter']).mean()[['result']]

result
concept year quarter          
1       2018 4        1.645833
        2019 2        1.755556
             4        1.872222
3       2018 2        1.694444
             4        1.537037
        2019 2        1.944444
             4        1.861111
4       2018 2        1.000000
             4        1.111111
        2019 2        1.333333
             4        1.555556

In [11]:
results_tidy.groupby(['pm_cat', 'year', 'quarter']).mean()[['result']]

result
pm_cat        year quarter          
Expect to See 2018 2        1.708333
                   4        1.645833
              2019 2        2.138889
                   4        2.203704
Like to See   2018 2        1.729167
                   4        1.531250
              2019 2        1.842593
                   4        1.953704
Love to See   2018 2        1.125000
                   4        1.437500
              2019 2        1.333333
                   4        1.342593

In [12]:
# import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

In [13]:
def update_plot(concept, district, pm_cat):
    
    if concept == 'All':
        concept = slice(None)
    if district == 'All':
        district = slice(None)
    if pm_cat == 'All':
        pm_cat = slice(None)
    
    idx = pd.IndexSlice
    y = results.loc[idx[slice(None), district, pm_cat], 'result']
    x = list(range(len(y)))
    x_labels = [f'{index[-2]} Q{index[-1]}' for index in 
                results.loc[idx[:, district, pm_cat], :].index]
    
    layout = go.Layout(
        title='Indicator 3.2',
        yaxis_title='rating',
        xaxis_tickvals=x,
        xaxis_ticktext=x_labels,
        yaxis_tickvals=[1,2,3],
        yaxis_ticktext=['L', 'M', 'H']
    )
     
    trace1=go.Scatter(x=x, y=y)
    
#     fig = go.Figure(layout=layout, data=[trace1])
    fig = make_subplots(rows=3, cols=1)
    fig.add_trace(go.Scatter(x=x, y=y), row=1, col=1)
    
    fig.update_yaxes(range=[.75,3.25])

    fig.show()

In [14]:
results.loc[idx[slice(None), 'Kitui County', slice(None)], ['result']]

result
concept district     pm_cat        year quarter          
4       Kitui County Expect to See 2018 2        1.000000
                                        4        1.333333
                                   2019 2        1.666667
                                        4        2.333333
                     Like to See   2018 2        1.000000
                                        4        1.000000
                                   2019 2        1.333333
                                        4        1.333333
                     Love to See   2018 2        1.000000
                                        4        1.000000
                                   2019 2        1.000000
                                        4        1.000000

In [15]:
select_concept = widgets.Dropdown(
options=[('One', 1), ('Three', 3), ('Four', 4)],
description='Concept')
select_concept

Dropdown(description='Concept', options=(('One', 1), ('Three', 3), ('Four', 4)), value=1)

In [16]:
results.index[0]

(1, 'Debre Birhan', 'Expect to See', 2019, 2)

In [17]:
select_district = widgets.Dropdown(
    options=df['district'].unique(),
    description='District'
)
select_district

Dropdown(description='District', options=('Kitui County', 'Kabarole Learning Alliance', 'Kumi District', 'Naka…

In [18]:
select_pm_cat = widgets.Dropdown(
options=np.append(df['pm_cat'].unique(), 'All'),
    description='PM Category'
)
select_pm_cat

Dropdown(description='PM Category', options=('Expect to See', 'Like to See', 'Love to See', 'All'), value='Exp…

In [19]:
widgets.interactive(update_plot, 
                    concept=select_concept, 
                    district=select_district,
                    pm_cat=select_pm_cat)

interactive(children=(Dropdown(description='Concept', options=(('One', 1), ('Three', 3), ('Four', 4)), value=1…

In [20]:
widgets.VBox([select_concept, select_district, select_pm_cat])

In [21]:
update_plot(slice(None), 'Kitui County', 'Expect to See')

In [22]:
from ipywidgets import interact

In [23]:
@interact(input1=widgets.IntSlider(1,1,3))
def update_plot(input1):
    return input1


interactive(children=(IntSlider(value=1, description='input1', max=3, min=1), Output()), _dom_classes=('widget…

In [24]:
filter1 = widgets.SelectMultiple(options=list(range(3)), value=(0,0), description='Option 1')

In [25]:
filter1

SelectMultiple(description='Option 1', index=(0, 0), options=(0, 1, 2), value=(0, 0))

In [26]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …

In [27]:
results_tidy.head()

,concept,district,pm_cat,year,quarter,result
0,1,Debre Birhan,Expect to See,2019,2,2.666667
1,1,Debre Birhan,Expect to See,2019,4,2.666667
2,1,Debre Birhan,Like to See,2019,2,1.500000
3,1,Debre Birhan,Like to See,2019,4,2.000000
4,1,Debre Birhan,Love to See,2019,2,1.666667


In [28]:
results_tidy['period'] = results_tidy['year'] + results_tidy['quarter']/10
results_tidy.head()

,concept,district,pm_cat,year,quarter,result,period
0,1,Debre Birhan,Expect to See,2019,2,2.666667,2019.2
1,1,Debre Birhan,Expect to See,2019,4,2.666667,2019.4
2,1,Debre Birhan,Like to See,2019,2,1.500000,2019.2
3,1,Debre Birhan,Like to See,2019,4,2.000000,2019.4
4,1,Debre Birhan,Love to See,2019,2,1.666667,2019.2


In [29]:
#group by concept
group = 'concept'
concept_results = results_tidy.groupby([group, 'pm_cat', 'period']).mean()[['result']]
concept_results.head()

result
concept pm_cat        period          
1       Expect to See 2018.4  1.833333
                      2019.2  2.366667
                      2019.4  2.400000
        Like to See   2018.4  1.437500
                      2019.2  1.600000

In [30]:
#group by district
group = 'district'
district_results = results_tidy.groupby([group, 'pm_cat', 'period']).mean()[['result']]
district_results.head()

result
district     pm_cat        period          
Debre Birhan Expect to See 2019.2  2.666667
                           2019.4  2.666667
             Like to See   2019.2  1.500000
                           2019.4  2.000000
             Love to See   2019.2  1.666667

In [53]:
group = 'district'
selector = 'Kitui County'

In [54]:
df = (results_tidy
      .groupby([group, 'pm_cat', 'period'])
      .mean()[['result']]
)

In [55]:
df.head()

result
district     pm_cat        period          
Debre Birhan Expect to See 2019.2  2.666667
                           2019.4  2.666667
             Like to See   2019.2  1.500000
                           2019.4  2.000000
             Love to See   2019.2  1.666667

In [56]:
y= df.loc[pd.IndexSlice[selector, 'Expect to See'], 'result']
y

period
2018.2    1.000000
2018.4    1.333333
2019.2    1.666667
2019.4    2.333333
Name: result, dtype: float64

In [57]:
data = []
for pm_cat in results_tidy['pm_cat'].unique():
    df = (results_tidy
         .groupby([group, 'pm_cat', 'period'])
         .mean()[['result']]
        )
    idx = pd.IndexSlice
    y= df.loc[idx[selector, pm_cat], 'result']
    
    x = list(range(len(y))) 

    trace = go.Scatter(x=x, y=y)

    data.append(trace)
data[2]

Scatter({
    'x': [0, 1, 2, 3], 'y': array([1., 1., 1., 1.])
})

In [58]:
x_tick_text = df.loc[pd.IndexSlice[selector, 'Expect to See'],:].index
x_tick_text

Float64Index([2018.2, 2018.4, 2019.2, 2019.4], dtype='float64', name='period')

In [59]:
#     fig = go.Figure(layout=layout, data=[trace1])
fig = make_subplots(rows=len(data), cols=1,
                   subplot_titles=results_tidy['pm_cat'].unique(),
                   shared_xaxes=True)

layout = go.Layout(
    title='Indicator 3.2',
    yaxis_title='rating',
#     xaxis_tickvals=x,
#     xaxis_ticktext=x_labels,
    yaxis_tickvals=[1,2,3],
    yaxis_ticktext=['L', 'M', 'H']
)

for idx, trace in enumerate(data):
    fig.add_trace(trace, row=idx+1, col=1)

# Update y-axes properties
fig.update_yaxes(range=[0.75,3.25], 
                 showgrid=False, 
                 tickvals=[1,2,3], 
                 ticktext=['L', 'M', 'H'])

# Update x-axes properties
x_tick_text = df.loc[pd.IndexSlice[selector, 'Expect to See'],:].index
fig.update_xaxes(tickvals=x, ticktext=x_tick_text)
fig.update_layout(showlegend=False)

fig.show()

In [60]:
def update_plot_group(group, selector):
    
#     if group == 'concept':
#         selector = concept
#     if group == 'district':
#         selector = district
    
    data = []
    for pm_cat in results_tidy['pm_cat'].unique():
        data_df = (results_tidy
          .groupby([group, 'pm_cat', 'period'])
          .mean()[['result']]
        )

        y = data_df.loc[pd.IndexSlice[selector, pm_cat], 'result']
        x = list(range(len(y))) 

        trace = go.Scatter(x=x, y=y)

        data.append(trace)
    
    x_tick_text = data_df.loc[pd.IndexSlice[selector, results_tidy['pm_cat'].unique()[0]], :].index
    
    fig = make_subplots(rows=len(data), cols=1,
                   subplot_titles=results_tidy['pm_cat'].unique(),
                   shared_xaxes=True)
    
    for idx, trace in enumerate(data):
        fig.add_trace(trace, row=idx+1, col=1)

    # Update y-axes properties
    fig.update_yaxes(range=[0.75,3.25], 
                     showgrid=False, 
                     tickvals=[1,2,3], 
                     ticktext=['L', 'M', 'H'])

    # Update x-axes properties
    x_tick_text = df.loc[pd.IndexSlice[selector, 'Expect to See'],:].index
    fig.update_xaxes(tickvals=x, ticktext=x_tick_text)
    fig.update_layout(showlegend=False)

    fig.show()

In [61]:
selector_dict = {
    'concept': [('One', 1), ('Three', 3), ('Four', 4)],
    'district': districts['district'].unique()
}

In [62]:
select_group = widgets.Dropdown(
    options=[('Concept', 'concept'), ('District', 'district')],
    description='Group By'
)
select_group

Dropdown(description='Group By', options=(('Concept', 'concept'), ('District', 'district')), value='concept')

In [63]:
selector_widget = widgets.Dropdown(
    options=selector_dict['concept'],
    value=1,
    description='Concept')
selector_widget

Dropdown(description='Concept', options=(('One', 1), ('Three', 3), ('Four', 4)), value=1)

In [64]:
type(selector_widget.value)

int

In [65]:
def on_update_group_widget(*args):
    selector_widget.options=selector_dict[select_group.value]
    selector_widget.description=select_group.value.capitalize()

select_group.observe(on_update_group_widget, 'value')

In [66]:
widgets.interactive(update_plot_group,
                    group=select_group,
                    selector=selector_widget)

interactive(children=(Dropdown(description='Group By', options=(('Concept', 'concept'), ('District', 'district…